In [ ]:
from torchvision import transforms
from datasets.Load_CelebA import CelebAData
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import random
import torch
def add_noise(input):
    noise=torch.randn((3,224,224))/256
    return input+noise
def paste_random_patch_tensor(image1, image2, patch_size=(100, 100)):
    """
    Paste a random patch from image2 to image1.

    :param image1: The image to paste the patch onto. Must be a PyTorch tensor of shape (C, H, W).
    :param image2: The image to take the patch from. Must be a PyTorch tensor of shape (C, H, W).
    :param patch_size: A tuple specifying the size of the patch (height, width).
    :returns: A new tensor which is image1 with the patch from image2 pasted onto it.
    """
    assert image1.shape == image2.shape, "Images must be the same size"
    _, height, width = image1.shape

    top = torch.randint(0, height - patch_size[0], size=(1,)).item()
    left = torch.randint(0, width - patch_size[1], size=(1,)).item()

    patch = image2[:, top:top+patch_size[0], left:left+patch_size[1]]
    image1[:, top:top+patch_size[0], left:left+patch_size[1]] = patch

    return image1


tranforms_train=transforms.Compose(
    [
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        transforms.RandomVerticalFlip(),
        transforms.RandomHorizontalFlip()
    ]
)
writer=SummaryWriter("log")

root="F:/CelebA_Spoof"
data=CelebAData(root_dir=root,train=True,transforms=tranforms_train)
image,label=data[0]
noisy_image=add_noise(image)

image1,label=data[1]

cut_image=paste_random_patch_tensor(image1=image,image2=image1)
writer.add_image(tag='cut_paste1',img_tensor=cut_image,global_step=0)
writer.add_image(tag='cut_paste1',img_tensor=image1,global_step=1)
writer.add_image(tag='cut_paste1',img_tensor=image,global_step=2)




writer.close()



In [5]:
from torch


torch.Size([256, 7, 7])
14336.0


In [ ]:
def cut_paste(self,input1,input2):
    width, height = 224, 224

    # 裁剪区域的最大宽度和高度
    max_patch_width, max_patch_height = 100, 100  # 你可以根据你的需要调整这些值

    # 随机生成裁剪区域的位置和大小
    x1 = random.randint(0, width - max_patch_width)
    y1 = random.randint(0, height - max_patch_height)
    w = random.randint(1, width - x1)
    h = random.randint(1, height - y1)

    # 裁切一块区域
    patch = input2[:, y1:y1 + h, x1:x1 + w]

    # 随机生成粘贴区域的位置
    x2 = random.randint(0, width - w)
    y2 = random.randint(0, height - h)

    # 将这块区域贴到另外一块区域
    input1[:, y2:y2 + h, x2:x2 + w] = patch
    return input1

In [ ]:
import torch
from torchvision import models
from torch import nn
model_resnet18=models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
model_resnet50=models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
text_student=nn.Sequential(*(list(models.resnet18(weights=models.ResNet18_Weights.DEFAULT).children())[:-1]))
text_student_head=text_student[0:4]
print(text_student_head)
# print(model_resnet18.layer1)
# print(model_resnet18.layer2)
# print(model_resnet18.layer3)
# print(model_resnet18.layer4)


In [ ]:
import torch
from torchvision import models
from loss import cosine_similarity_loss
from datasets.Load_CelebA import CelebAData
from networks.S_T_model import train_difference_extraction_model
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter
from configs import parse
model = train_difference_extraction_model()
train_transform = transforms.Compose(
            [
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                transforms.RandomVerticalFlip(),
                transforms.RandomHorizontalFlip()
            ]
        )
args = parse().parse_args()
# writer=SummaryWriter('testlog')
# print(args.data_dir)
# data=CelebAData(root_dir="F:/CelebA_Spoof",train=True,transforms=train_transform)
#
# image,label=data[0]
#
#
# image_noise=model.add_noise(image)
# image_cut=model.cut_paste(image)
#
# writer.add_image("test",image,0)
# writer.add_image("test",image_noise,1)
# writer.add_image("test",image_cut)
#
# text_normal_feature,text_noise_feature,difference_normal_featue,difference_cutpaste_featue=model(image)
#
# loss=cosine_similarity_loss(text_normal_feature,text_noise_feature)
# print(loss)
#
# loss=cosine_similarity_loss(difference_normal_featue,difference_cutpaste_featue)
#
# writer.close()




In [ ]:

import torch
from torchvision import models
from loss import cosine_similarity_loss
from datasets.Load_CelebA import CelebAData
from networks.S_T_model import train_difference_extraction_model
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter
from configs import parse
train_transform = transforms.Compose(
            [
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ]
        )
args = parse().parse_args()
writer=SummaryWriter('testlog')
data=CelebAData(root_dir="F:/CelebA_Spoof",train=True,transforms=train_transform,only_true=True)
dataloader=DataLoader(data,batch_size=8,shuffle=True)

step=0
for i in dataloader:
    images,labels=i
    writer.add_images("True",images,step)
    step+=1
    if step==20:
        break

writer.close()





In [ ]:
import os
root="F:/CelebA_Spoof/metas/intra_test"
txt=os.path.join(root,'test_label.txt')
image_paths=[]
labels=[]
with open(txt,'r') as f:
    for line in f:
        path,label=line.rstrip('\n').split(' ')
        print(f'path:{path}      label:{label}')



In [ ]:
t='Train_1'
if 'Train' in t:
    img_use='Train_files'
elif 'Test' in t:
    img_use='Test_files'
elif 'Dev' in t:
    img_use='Dev_files'


In [ ]:
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
from datasets.Load_CelebA import CelebAData
from torchvision import transforms
from networks.R50_model import R50
from networks.SwinV2_model import SwinV2
import torch.optim as optim
import argparse
from torch.nn import functional as F
from configs import parse
from utils.performance import test_accuracy
from networks.S_T_model import train_difference_extraction_model
from  loss import cosine_similarity_loss
args = parse().parse_args()
train_transform=transforms.Compose(
        [
            transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225]),
            transforms.RandomVerticalFlip(),
            transforms.RandomHorizontalFlip()
        ]

    )
test_transform=transforms.Compose(
    [
        transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225]),
        transforms.RandomVerticalFlip(),
        transforms.RandomHorizontalFlip()
    ]
)
train_data=CelebAData(root_dir=args.data_dir,train=True,transforms=train_transform)
test_data=CelebAData(root_dir=args.data_dir,train=False,transforms=test_transform)
train_data_true=CelebAData(root_dir=args.data_dir,train=True,transforms=train_transform,only_true=True)
train_loader=DataLoader(train_data,batch_size=args.batch_size,shuffle=True)
test_loader=DataLoader(train_data,batch_size=args.batch_size,shuffle=True)
train_true_loader=DataLoader(train_data_true,batch_size=args.batch_size,shuffle=True)

def cut_paste_single(self, image1,image2, max_patch_size=(100, 100)):
    # cut and paste image

    assert image1.shape == image2.shape
    _, height, width = image1.shape

    patch_height = random.randint(1, min(height, max_patch_size[0]) + 1)
    patch_width = random.randint(1, min(width, max_patch_size[1]) + 1)

    top = random.randint(0, height - patch_height + 1)
    left = random.randint(0, width - patch_width + 1)
    patch = image2[:, top:top + patch_height, left:left + patch_width]
    # 定义一个高斯模糊核
    gaussian_kernel = torch.tensor([[1., 2, 1], [2, 4, 2], [1, 2, 1]]) / 16.0
    gaussian_kernel = gaussian_kernel.view(1, 1, 3, 3).repeat(3, 1, 1, 1)  # Repeat for each channel

    # 对覆盖图像进行高斯模糊
    overlay_blurred = F.conv2d(patch.unsqueeze(0), gaussian_kernel, padding=1).squeeze(0)

    result = image1.clone()
    result[:, top:top + patch_height, left:left + patch_width] = overlay_blurred
    return result


image1,_=train_data[0]
image2,_=train_data[1]
writer=SummaryWriter("testlog")

writer.add_image("cut",image1,0)
result=cut_paste_single(image1,image2)
writer.add_image("cut",result,1)
writer.add_image("cut",image2,2)
writer.close()



In [ ]:
import os
import cv2
import dlib
import fnmatch
from concurrent.futures import ThreadPoolExecutor
import random
def process_video( video_path, root_dir, p, t, label, img_use):
    detector = dlib.get_frontal_face_detector()
    video = cv2.VideoCapture(os.path.join(root_dir, img_use, video_path))
    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    random_frames = random.sample(range(frame_count), 5)
    for frame_number in random_frames:
        video.set(1, frame_number)
        ret, frame = video.read()
        video_name = video_path.split('.')[0]
        img_path = os.path.join(root_dir, p, t.split('.')[0], label, f'{video_name}_{frame_number}.png')
        if ret:
            dets = detector(frame, 1)
            if not dets:
                print(f'Failed to detector face {p}_{t}_{label}_{video_path}')
            for i, d in enumerate(dets):
                x1 = d.top() if d.top() > 0 else 0
                y1 = d.bottom() if d.bottom() > 0 else 0
                x2 = d.left() if d.left() > 0 else 0
                y2 = d.right() if d.right() > 0 else 0

                if not os.path.exists(os.path.dirname(img_path)):
                    try:
                        os.makedirs(os.path.dirname(img_path))
                    except:
                        print('error')

                face = frame[x1:y1,x2:y2]
                w,h,c=face.shape()
                if w<100 and h<100 and c<100:
                    print("detect fial")
                    continue
                cv2.imwrite(img_path, face)
        else:
            print(f'Failed to read frame {video_name}')
    video.release()

def process_data(root_dir):
    protocol = ['Protocol_1', 'Protocol_2', 'Protocol_3', 'Protocol_4']
    for p in protocol:
        pro_txt = []
        for root, dirs, files in os.walk(os.path.join(root_dir, 'Protocols', p)):
            for file in files:
                if fnmatch.fnmatch(file, '*.txt'):
                    pro_txt.append(file)
        for t in pro_txt:
            with open(os.path.join(root_dir, 'Protocols', p, t), 'r') as file:
                lines = file.readlines()
            video_paths = []
            labels = []
            face_txt_paths = []
            for line in lines:
                parts = line.strip().split(',')
                label = parts[0]
                path = parts[1]
                video_path = path + '.avi'
                video_paths.append(video_path)
                labels.append(label)


            if 'Train' in t:
                img_use = 'Train_files'
            elif 'Test' in t:
                img_use = 'Test_files'
            elif 'Dev' in t:
                img_use = 'Dev_files'
            with ThreadPoolExecutor(max_workers=4) as executor:
                for num in range(len(video_paths)):
                    executor.submit(process_video, video_paths[num], root_dir, p, t, labels[num], img_use)



root_dir='E:\\OULU-NPU'
process_data(root_dir)

In [2]:
import torch
import timm
from torchsummary import summary

model = timm.create_model('swin_base_patch4_window7_224', pretrained=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# 创建一个随机输入张量，用于获取模型的输入大小
input_size = (3, 224, 224)  # 根据模型期望的输入尺寸设置
input_tensor = torch.randn(1, *input_size).to(device)

# 使用 torchsummary 打印模型信息
summary(model, input_size=input_size)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 128, 56, 56]           6,272
         LayerNorm-2          [-1, 56, 56, 128]             256
        PatchEmbed-3          [-1, 56, 56, 128]               0
          Identity-4          [-1, 56, 56, 128]               0
         LayerNorm-5          [-1, 56, 56, 128]             256
            Linear-6              [-1, 49, 384]          49,536
           Softmax-7            [-1, 4, 49, 49]               0
           Dropout-8            [-1, 4, 49, 49]               0
            Linear-9              [-1, 49, 128]          16,512
          Dropout-10              [-1, 49, 128]               0
  WindowAttention-11              [-1, 49, 128]               0
         Identity-12          [-1, 56, 56, 128]               0
        LayerNorm-13            [-1, 3136, 128]             256
           Linear-14            [-1, 31

In [ ]:
import torchvision

model=torchvision.models.swin_transformer.SwinTransformer.swin_t